In [1]:
from __future__ import print_function
import bqapi as bp
import pandas
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

To retrieve data, one needs to obtain a session object. This is to allow multiple sessions in a program, and to avoid global state within bqapi altogether. The API developer's guide lists some reasons where multiple sessions might be desirable as well.

In [2]:
sess = bp.Session()

# Reference Data

Retrieve data synchronously. By default, a `pandas.DataFrame` is returned.

In [3]:
sess.get_data('ibm us equity', 'px last')

Field,px last
Security,
ibm us equity,150.72


The function will operate asynchronously if invoked with a `callback` parameter. It returns immediately with a promise for the result. The `result()` call can be used to wait until the result is available.

In [4]:
def id(x):
    return x
promise = sess.get_data('ibm us equity', 'px_last', callback=id)
promise.result()

Field,px_last
Security,
ibm us equity,150.72


If an error occurs, the exception is propagated

In [5]:
sess.get_data('ibm us equity', 'px_las', callback=id).result()

FieldError: Failed to process reply for security "ibm us equity", field "px_las": Field not valid

Asynchronous requests allow to send multiple requests in parallel, instead of having to wait for a server reply before sending the next request. The following sends three requests in parallel, and prints their result. This is somewhat pointless for simple reference data requests since one can simply query all three prices with a single request, however for other types of requests, such as historical intraday requests, only data from a single security can be queried with a single request.

In [6]:
promises = [sess.get_data(x, 'px_last', callback=id) for x in ['ibm us equity', 'goog us equity', 'aapl us equity']]
for promise in promises:
    df = promise.result()
    print('{}: {}'.format(df.index[0], df.ix[0,0]))

ibm us equity: 150.72
goog us equity: 795.26
aapl us equity: 117.47


The same data can be retrieved using a single request. Note that the order of the results might be different from the asynchronous example above.

In [7]:
df = sess.get_data(['ibm us equity', 'goog us equity', 'aapl us equity'], 'px last')
for security, price in zip(df.index, df['px last']):
    print('{}: {}'.format(security, price))

aapl us equity: 117.47
goog us equity: 795.26
ibm us equity: 150.72


# Promise callbacks and chains

Promises can issue callbacks with the `then()` function to execute a function once the requested data is available. As an example, here, extract the numerical value of the IBM price without the dictionary around it.

In [8]:
sess.get_data('ibm us equity', 'px last', callback=id).then(lambda x: float(x['px last'][0])).result()

150.72

Another possibility is using aggregators such as `sum()`:

In [9]:
sess.get_data('ibm us equity', 'px last',callback=pandas.DataFrame.sum).then(pandas.Series.sum).then(float).result()

150.72

If the original `get_data()` call fails, the exception is propagated to the promise returned.

In [10]:
sess.get_data('ibm us equity', 'px las', callback=lambda x: x['px last'][0]).result()

FieldError: Failed to process reply for security "ibm us equity", field "px las": Field not valid

If the return value of the callback function is another promise, then the original promise will hold the result of the returned promise once it becomes available. This allows to chain promises. Here, we first request one field, and once it is available, we request two more fields and merge them with the first result. Note that `get_more_data()` returns a promise, and therefore `.then(get_more_data)` does not wait until `get_more_data()` itself returns, but until the promise returned by `get_more_data()` is available.

If at any point during the chain an error occurs, it is propagated correctly to the promise holding the final value.

In [11]:
def get_more_data(orig_data):
    # Note this returns a promise
    return sess.get_data('ibm us equity', ['px high', 'px low'],
                         callback=lambda data: pandas.concat([orig_data, data], axis=1))

sess.get_data('ibm us equity', 'px last', callback=get_more_data).result()

Field,px last,px high,px low
Security,,,
ibm us equity,150.72,151.0,147.79


# Real-Time Subscriptions

The `subscribe()` method allows to perform subscriptions to real-time fields. A subscription object is returned that can be used to register a callback for subscription updates and also to unsubscribe later.

In [12]:
def update(data):
    print(data.ix[0, 0])

subscription = sess.subscribe('ibm us equity', 'last_price')
subscription.on_update().then(update)

Run the event loop for 5 seconds, to see some price updates.

In [13]:
bp.after(5).then(lambda _: bp.stop())
bp.start()

150.72


The subscription can be canceled by calling `Session.unsubscribe()`.

In [14]:
sess.unsubscribe(subscription)

# Output Format

The `format` parameter of `get_data()` and `subscribe()` can be used to specify the format of the data output. At the moment, output as a list of python dictionaries, `pandas.DataFrame` and matplotlib or bqplot plots are supported. It is also possible to add custom formatters.

In [15]:
sess.get_data(['ibm us equity', 'goog us equity', 'aapl us equity'], 'px last', format=bp.dictionary)

[{'Security': 'ibm us equity', 'px last': 150.72},
 {'Security': 'goog us equity', 'px last': 795.26},
 {'Security': 'aapl us equity', 'px last': 117.47}]

In [16]:
sess.get_data(['ibm us equity', 'goog us equity', 'aapl us equity'], 'px last', format=bp.data_frame)

Field,px last
Security,
aapl us equity,117.47
goog us equity,795.26
ibm us equity,150.72


# Low-Level API

The `send_request()` and `send_subscribe()` functions allow to send arbitrary requests to any service. No processing on the output data returned by the API is performed, since we don't know how to interpret it. The returned structures are thin wrappers around the BLPAPI elements and mostly compatible with how dictionaries and lists work, and can be converted to them as well.

In [17]:
result = sess.send_request('//blp/refdata', 'ReferenceDataRequest',
                           {'securities': ['ibm us equity'], 'fields': ['px last']})

In [18]:
result

[{'securityData': [{'security': 'ibm us equity', 'fieldData': {'px last': 150.72}, 'fieldExceptions': [], 'sequenceNumber': 0, 'eidData': []}]}]

In [19]:
result[0]['securityData'][0]['fieldData'][0]

150.72

# Global Callbacks

For particular events, it is possible to register global callbacks

In [20]:
def on_session_terminated(session, exc_info):
    print('The session has been terminated!')

sess.register_callback('SessionTerminated', on_session_terminated)

Terminating the session will now run the callback. In particular, this also happens when the session is terminated asynchronously, for example if the network connection goes down.

In [21]:
sess.close()

The session has been terminated!


The following events are defined at the moment:
    
  * `SessionStarted(session)`
  * `SessionTerminated(session, exc_info)`
  * `ServiceOpened(session, service_name, blpapi_service)`
  * `SubscriptionStarted(session, subscription)`
  * `SubscriptionTerminated(session, subscription)`
  * `MarketDataEvents(session, security, fields, values)`

To the extent possible under law, Bloomberg L.P. has waived all copyright and related or neighboring rights to this notebook, in accordance with the Creative Commons CC0 1.0 Public Domain Dedication ([http://creativecommons.org/publicdomain/zero/1.0/](http://creativecommons.org/publicdomain/zero/1.0/)).